In [1]:
import pandas as pd
import os
import duckdb
from deep_translator import GoogleTranslator

from dotenv import load_dotenv

# load .env file
env_file_path = "../.env"
load_dotenv(dotenv_path=env_file_path)
path = os.getenv("HOME_PATH")


In [2]:
#csv_path = path + '/data/raw/utf8_data.csv'
#
#data = pd.read_csv(csv_path)
#
## Inicializar o tradutor
#translator = GoogleTranslator(source='en', target='pt')
#
#def traduzir_texto(texto):
#    try:
#        # Traduzir o texto para português
#        traducao = translator.translate(texto)
#        return traducao.text
#    except Exception as e:
#        print(f"Erro ao traduzir texto: {texto}. Erro: {e}")
#        return texto  # Retorna o texto original em caso de erro
#
## Aplicar a função de tradução a cada elemento da coluna 'Chief_complain'
#data['Chief_complain_pt'] = data['Chief_complain'].apply(traduzir_texto)
#
## Devido às limitações de chamadas para a API, pode ser útil adicionar um atraso entre as traduções
##time.sleep(1)
#
## Verificar as primeiras traduções
#print(data[['Chief_complain', 'Chief_complain_pt']].head())

In [3]:
dbpath = path + "/data/interim/interim.db"

conn = duckdb.connect(dbpath)

In [5]:
query = """SELECT (Diagnosis_in_ED) FROM triagem LIMIT 10;"""
result = conn.execute(query).fetchdf()

BinderException: Binder Error: Referenced column "Diagnosis_in_ED" not found in FROM clause!
Candidate bindings: "triagem.Diagnosis in ED"
LINE 1: SELECT (Diagnosis_in_ED) FROM triagem LIMIT 10;
                ^

In [4]:
# creating a new column in the table
conn.execute("ALTER TABLE triagem ADD COLUMN Chief_complain_pt TEXT;")

# google translator
translator = GoogleTranslator(source='en', target='pt')

query = """SELECT Chief_complain
            FROM triagem"""
            
result = conn.execute(query).fetchall()

for row in result:
    text = row[0] # adjust the index to match the column you want to translate
    try:
        translation = translator.translate(text)
        # update the table with the translation
        update_query = f"UPDATE triagem SET Chief_complain_pt = ? WHERE Chief_complain = ?;"
        # execute the query
        conn.execute(update_query, (translation, text))
    except Exception as e:
        print(f"Erro ao traduzir o text {text}. Error: {e}")
        
conn.commit()




In [8]:
result

,Group,Sex,Age,Patients number per hour,Arrival mode,Injury,Chief_complain,Mental,Pain,NRS_pain,...,Saturation,KTAS_RN,Diagnosis in ED,Disposition,KTAS_expert,Error_group,Length of stay_min,KTAS duration_min,mistriage,Chief_complain_pt
0,2,2,71,3,3,2,right ocular pain,1,1,2,...,100,2,Corneal abrasion,1,4,2,86,"5,00",1,dor ocular direita
1,1,1,56,12,3,2,right forearm burn,1,1,2,...,None,4,"Burn of hand, firts degree dorsum",1,5,4,64,"3,95",1,queimadura no antebraço direito
2,2,1,68,8,2,2,"arm pain, Lt",1,1,2,...,98,4,"Fracture of surgical neck of humerus, closed",2,5,4,862,"1,00",1,"dor no braço, tenente"
3,1,2,71,8,1,1,ascites tapping,1,1,3,...,None,4,Alcoholic liver cirrhosis with ascites,1,5,6,108,"9,83",1,ascite batendo
4,1,2,58,4,3,1,"distension, abd",1,1,3,...,None,4,Ascites,1,5,8,109,"6,60",1,"distensão, abd"
5,2,1,54,6,4,1,fever,1,1,3,...,98,3,"Fever, unspecified",2,4,1,9246,"2,00",1,febre
6,2,2,49,11,3,1,With chest discomfort,1,1,3,...,98,2,"Angina pectoris, unspecified",1,3,2,400,"3,00",1,Com desconforto no peito
7,1,2,78,14,3,1,"pain, chest",1,1,3,...,None,2,Acute coronary syndrome,1,3,2,247,"10,23",1,"dor, peito"
8,1,2,32,10,3,1,LBP - Low back pain,1,1,3,...,None,4,Herniated disc disease of lumbar spine with ra...,1,5,4,59,"3,23",1,LBP – Dor lombar
9,2,1,38,6,3,1,"Eczema, Eyelid",1,1,3,...,97,4,Ocular pain,1,5,4,185,"4,00",1,"Eczema, Pálpebra"
